<a href="https://colab.research.google.com/github/mhsh77/Martingale-eurusd/blob/main/Martingale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
!pip install pandas_ta

In [42]:
!pip install backtesting

In [43]:
import pandas as pd
import pandas_ta as ta
import numpy as np
from scipy.stats import linregress
from matplotlib import pyplot
from backtesting import Strategy,Backtest

In [44]:
df = pd.read_csv('/content/drive/MyDrive/trading/EURUSD_Candlestick_4_Hour_BID_01.12.2020-14.12.2024.csv')
df.tail()

,Local time,Open,High,Low,Close,Volume
8850,14.12.2024 05:30:00.000 GMT+0330,1.05017,1.05017,1.05017,1.05017,0.0
8851,14.12.2024 09:30:00.000 GMT+0330,1.05017,1.05017,1.05017,1.05017,0.0
8852,14.12.2024 13:30:00.000 GMT+0330,1.05017,1.05017,1.05017,1.05017,0.0
8853,14.12.2024 17:30:00.000 GMT+0330,1.05017,1.05017,1.05017,1.05017,0.0
8854,14.12.2024 21:30:00.000 GMT+0330,1.05017,1.05017,1.05017,1.05017,0.0


In [45]:
df = df[df['Volume']!=0]
df.reset_index(drop=True,inplace=True)
df.isna().sum()

,0
Local time,0
Open,0
High,0
Low,0
Close,0
Volume,0


#Generating signals randomly

In [46]:
df['Signal'] = np.random.choice([1,2], size=len(df))#here probibilty can be added and the probibilty can be the scope of trend
df.rename(columns={'Local time':'Time'})
df.tail()

,Local time,Open,High,Low,Close,Volume,Signal
6302,13.12.2024 05:30:00.000 GMT+0330,1.04652,1.04667,1.04562,1.04652,11893.95,1
6303,13.12.2024 09:30:00.000 GMT+0330,1.04653,1.04940,1.04531,1.04871,26114.92,1
6304,13.12.2024 13:30:00.000 GMT+0330,1.04873,1.05241,1.04834,1.05102,52522.74,2
6305,13.12.2024 17:30:00.000 GMT+0330,1.05102,1.05103,1.04822,1.04963,47795.54,1
6306,13.12.2024 21:30:00.000 GMT+0330,1.04964,1.05045,1.04891,1.05017,24674.05,1


##Create the strategy

In [63]:
stop=300e-4
profit=300e-4
def SIGNAL():
  return df.Signal
class Martingale(Strategy):
  initalvalue = 1000
  ordersize = initalvalue
  def init(self):
    super().init()
    self.signal = self.I(SIGNAL)
  def next(self):
    super().next()
    # check if the signal is + and we have no open trades it is not our first trade and we lost the first trade
    if(self.signal>0 and len(self.trades)==0 and len(self.closed_trades)>0 and self.closed_trades[-1].pl<0):
      self.ordersize*=2
      #if the signal
    elif len(self.closed_trades)>0 and self.closed_trades[-1].pl>0:
      self.ordersize = self.initalvalue
      #buy
    if self.signal==2 and len(self.trades)==0:
      sl = self.data.Close[-1] - stop
      tp = self.data.Close[-1] + profit
      self.buy(sl=sl,tp=tp,size=self.ordersize)
      #sell
    elif self.signal==1 and len(self.trades)==0:
      sl = self.data.Close[-1] + stop
      tp = self.data.Close[-1] - profit
      self.sell(sl=sl,tp=tp,size=self.ordersize)



In [65]:
initalvalue = 10000
backtest = Backtest(df, Martingale,cash=initalvalue,commission=0.00)
stat = backtest.run()
backtest.plot()

<ipython-input-65-f0d4d9750fb5>:2: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  backtest = Backtest(df, Martingale,cash=initalvalue,commission=0.00)


GridPlot(id='p6450', ...)

In [49]:
stat

,0
Start,0.0
End,6306.0
Duration,6306.0
Exposure Time [%],99.968289
Equity Final [$],10003.4184
Equity Peak [$],10003.6777
Return [%],0.034184
Buy & Hold Return [%],-11.93986
Return (Ann.) [%],0.0
Volatility (Ann.) [%],NaN
